## Attractor Landscape analysis

This is an example of how Boolean network model was analyzed in our paper.

We provide the codes for 'Attractor landscape analysis', 'Perturbation analysis', and 'Analysis of network dynamics' (Please refer to the Method section).


In [ ]:
import numpy as np
import pandas as pd
import itertools
import networkx as nx
import copy
import os
from collections import defaultdict, deque

from pyboolnet.file_exchange import bnet2primes, primes2bnet
from pyboolnet.interaction_graphs import primes2igraph
from pyboolnet.state_transition_graphs import primes2stg
from pyboolnet.attractors import compute_attractors_tarjan

from modules.attractorSim import rand_initial_states, compute_attractor_from_primes, compute_phenotype, Simulation
from modules.printlogic import print_node_logic

In [ ]:
network_dir = './network/'
model_file = network_dir + 'random_network.bnet'
primes = bnet2primes(model_file)
nodeList = list(primes.keys())
# nodeList = sorted(primes.keys(), key=lambda s: int(s[len("Node"):]))
graph = primes2igraph(primes)
update_mode = "synchronous"

# nodes_order = sorted(primes.keys(), key=lambda s: int(s[len("Node"):]))
nodes_order = sorted(primes.keys())
print("총 노드 수:", len(nodes_order))
# print("앞 31개 노드:", nodes_order[:31])
print("진짜 앞 31개 노드:", nodeList[:31])

# ----------------------------------------------------
# 랜덤 네트워크를 위한 Phenotype/Markers 정의 (EMT 네트워크 기준 재사용)
# ----------------------------------------------------

# 1. 가상의 마커 노드 선택
# ER 네트워크의 노드 중에서 2개를 선택 (첫 번째와 두 번째 노드)
if len(nodes_order) < 2:
    raise ValueError("랜덤 네트워크의 노드가 너무 적어서 마커를 지정할 수 없습니다.")

virtual_Ecad_node = nodes_order[0] # Node1 (보통 Node0/Node1 등으로 시작할 것임)
virtual_ZEB_node = nodes_order[1]  # Node10
# virtual_ZEB_node = nodes_order[9]  # Node10

print(f"가상 E-cad 노드: {virtual_Ecad_node}, 가상 ZEB1 노드: {virtual_ZEB_node}")

# 2. markers 변수 (기존 markers 변수 대체)
markers_er = [virtual_Ecad_node, virtual_ZEB_node]

# 3. phenotype 정의 (EMT 논문 기준과 동일하게)
phenotype_er = {
    'P_pheno_1': {virtual_Ecad_node:1, virtual_ZEB_node:0},   # epithelial
    'P_pheno_2': {virtual_Ecad_node:0, virtual_ZEB_node:1},  # mesenchymal
    'P_pheno_3': {virtual_Ecad_node:1, virtual_ZEB_node:1},  # hybrid (E-cad + / ZEB1 +)
    'P_pheno_4' : {virtual_Ecad_node:0, virtual_ZEB_node:0}   # hybrid (E-cad - / ZEB1 -)
}

# 4. phenotypeAnnot 정의 (EMT 논문 기준과 동일하게)
phenotypeAnnot_er = {
    'P_pheno_1': -1,
    'P_pheno_2': 1,
    'P_pheno_3': 0,
    'P_pheno_4': 0
}

print("랜덤 네트워크를 위한 phenotype/markers 정의 완료.")
# --- 끝 ---


if 2**len(nodeList) >= 100000: num_init = 100000
else:  num_init = 2**len(nodeList)
initState = rand_initial_states(num_init, len(nodeList))

총 노드 수: 31
진짜 앞 31개 노드: ['Node1', 'Node10', 'Node11', 'Node12', 'Node13', 'Node14', 'Node15', 'Node16', 'Node17', 'Node18', 'Node19', 'Node2', 'Node20', 'Node21', 'Node22', 'Node23', 'Node24', 'Node25', 'Node26', 'Node27', 'Node28', 'Node29', 'Node3', 'Node30', 'Node31', 'Node4', 'Node5', 'Node6', 'Node7', 'Node8', 'Node9']
가상 E-cad 노드: Node1, 가상 ZEB1 노드: Node10
랜덤 네트워크를 위한 phenotype/markers 정의 완료.


### Node perturbation analysis

In [3]:
save_dir = './result/EMT/' 
save_perturbname = save_dir + 'Random_simul_result.csv'

In [4]:
fix_dict = {} # 또는 가상의 RAS 노드 고정

In [5]:
fix_dict_tmp = copy.deepcopy(fix_dict) # fix_dict는 {'RAS':1} 인 상태

print(fix_dict_tmp) # 현재 적용할 고정 노드 출력

primes_new, pheno_df, att_ave_pd, attrs_dict = Simulation(fix_dict_tmp, primes, update_mode, initState, phenotype_er, phenotypeAnnot_er)   

print(f'attrs_dict={attrs_dict}')


ori_attrs_num = len(attrs_dict)
print(f'어트랙터 개수: {ori_attrs_num}')


{}
WARNING The state transition graph will consist of up to 2**31=2147483648 states, depending on the initial states.

--- [Simulation Function] Detected Attractors for Current Perturbation ---
  > Attractor ID 0 (Steady State, Basin 51.10%)
    State Str:  1000001110010111100000010010111
  > Attractor ID 1 (Steady State, Basin 5.22%)
    State Str:  1101111100110000100000001010010
  > Attractor ID 2 (Steady State, Basin 21.08%)
    State Str:  1000111100110000100000000010010
  > Attractor ID 3 (Cyclic, Length 2, Basin 22.60%)
    Cycle Start Str:  1101111100110000100000000010010
       Step 1 Str:  1101111100110000100000000010010
       Step 2 Str:  1000111100110000100000001010010
--- End of Attractor Listing ---
Attractor simulation time : 10.746366500854492
              Ratio
phenotype          
P_pheno_1  0.721782
P_pheno_3  0.278218
attrs_dict=defaultdict(None, {0: defaultdict(None, {'attractors': ('1000001110010111100000010010111',), 'basinsizes': 66409, 'perc': 0.51095637454797

### 하나씩 없앤 네트워크 분석

In [6]:
def remove_char_at_index(s: str, index: int) -> str:
    """
    주어진 문자열에서 특정 인덱스의 문자를 제거합니다.
    """
    if not (0 <= index < len(s)):
        raise IndexError(f"인덱스 {index}가 문자열 길이 {len(s)}의 범위를 벗어납니다.")
    return s[:index] + s[index+1:]

def transform_attractor_for_comparison(attractor_tuple: tuple, removed_node_index: int) -> tuple:
    """
    original attractor 튜플의 각 문자열에서 removed_node_index에 해당하는 문자를 제거하여
    비교를 위한 새로운 튜플을 생성합니다.
    """
    transformed_strings = []
    for s in attractor_tuple:
        transformed_strings.append(remove_char_at_index(s, removed_node_index))
    return tuple(transformed_strings)

def are_cyclic_attractors_identical(attractor1, attractor2):
    """
    두 개의 cyclic attractor 튜플이 순환적으로 동일한지 비교합니다.
    """
    if len(attractor1) != len(attractor2):
        return False

    if not attractor1 or not attractor2: # 빈 튜플인 경우
        return False

    d1 = deque(attractor1)
    
    for _ in range(len(attractor1)):
        if tuple(d1) == attractor2:
            return True
        d1.rotate(1) # 한 칸씩 회전

    return False

def compare_attractors(attr_tuple1, attr_tuple2):
    """
    두 개의 attractor 튜플을 비교하여 동일한지 여부를 반환합니다.
    steady-state와 cyclic attractor를 구분하여 비교합니다.
    """
    len1 = len(attr_tuple1)
    len2 = len(attr_tuple2)

    if len1 == 1 and len2 == 1:
        return attr_tuple1[0] == attr_tuple2[0]
    elif len1 > 1 and len2 > 1:
        # ↓↓↓ 여기가 문제의 발생 원인이었습니다. attr_tuple1, attr_tuple2로 수정했습니다. ↓↓↓
        return are_cyclic_attractors_identical(attr_tuple1, attr_tuple2)
        # ↑↑↑
    else:
        return False

def find_identical_attractors(original_data, new_data, removed_node_index: int):
    """
    original 네트워크와 new 네트워크 간에 동일한 attractor를 찾아 출력합니다.
    original 네트워크의 각 attractor는 removed_node_index에 해당하는 노드를 제거한 후 비교합니다.
    """
    print(f"--- Attractor 비교 결과 (original에서 노드 {removed_node_index} 제거 후) ---")
    found_any_match = False

    for original_key, original_info in original_data.items():
        original_attractor_raw = original_info['attractors']
        
        try:
            if not original_attractor_raw:
                print(f"⚠️ Original {original_key}번 어트랙터 튜플이 비어있습니다. 건너뜁니다.")
                continue
            
            # 모든 문자열의 길이가 removed_node_index보다 길어야만 제거를 시도
            if any(len(s) <= removed_node_index for s in original_attractor_raw):
                 print(f"⚠️ Original {original_key}번 어트랙터 문자열 중 노드 {removed_node_index}를 제거할 수 없는 짧은 문자열이 있습니다. 건너뜁니다.")
                 continue

            original_attractor_transformed = transform_attractor_for_comparison(original_attractor_raw, removed_node_index)
            original_type = "Steady-state" if len(original_attractor_transformed) == 1 else "Cyclic"

        except IndexError as e:
            print(f"⚠️ Original {original_key}번 어트랙터 변형 중 오류 발생 (인덱스 {removed_node_index}): {e}. 이 어트랙터는 건너뜁니다.")
            continue
        
        match_found_for_this_original = False
        
        for new_key, new_info in new_data.items():
            new_attractor = new_info['attractors']
            new_type = "Steady-state" if len(new_attractor) == 1 else "Cyclic"

            if not original_attractor_transformed or not new_attractor:
                continue 
            
            # 변형된 original 어트랙터와 new 어트랙터의 문자열 길이가 다르면 비교하지 않음
            # 모든 문자열이 동일한 길이를 가지는 것을 가정
            if len(original_attractor_transformed[0]) != len(new_attractor[0]):
                continue

            if compare_attractors(original_attractor_transformed, new_attractor):
                print(f"✅ 동일한 attractor를 찾았습니다!")
                print(f"  - Original: {original_key}번 (원본: {original_attractor_raw}, {original_type})")
                print(f"    (노드 {removed_node_index} 제거 후): {original_attractor_transformed}")
                print(f"  - New     : {new_key}번 ({new_type})")
                print(f"    어트랙터: {new_attractor}")
                print("-" * 30)
                found_any_match = True
                match_found_for_this_original = True
                break
        
        if not match_found_for_this_original:
            print(f"❌ Original {original_key}번 어트랙터 (노드 {removed_node_index} 제거 후)는 New 어트랙터에서 일치하는 것을 찾지 못했습니다.")
            print(f"   (노드 제거 전): {original_attractor_raw}")
            print(f"   (노드 제거 후): {original_attractor_transformed}")
            print("-" * 30)

    if not found_any_match:
        print(f"💡 Original과 New 네트워크 사이에 (노드 {removed_node_index} 제거 후) 동일한 attractor를 찾을 수 없습니다.")

In [8]:
remove_network_dir = './network/random_network/random_network_'
remove_fix_dict = copy.deepcopy(fix_dict) # fix_dict는 {} 인 상태
remove_update_mode = "synchronous"

for i in range(len(nodes_order)):
    remove_node_num = int(nodes_order[i][4:])
    print(f'------remove logic of {nodes_order[i]}------')
    try:
        remove_model_file = remove_network_dir + str(remove_node_num) + '.bnet'
        print(remove_model_file)
        remove_primes = bnet2primes(remove_model_file)
        print(remove_fix_dict) # 현재 적용할 고정 노드 출력

        remove_nodeList = list(remove_primes.keys())
        if 2**len(remove_nodeList) >= 100000: remove_num_init = 100000
        else:  remove_num_init = 2**len(remove_nodeList)
        remove_initState = rand_initial_states(remove_num_init, len(remove_nodeList))
        
        reomve_primes_new, remove_pheno_df, remove_att_ave_pd, remove_attrs_dict = Simulation(remove_fix_dict, remove_primes, remove_update_mode, remove_initState, phenotype_er, phenotypeAnnot_er)
        # print(f'\nremove_attrs_dict={remove_attrs_dict}\n')

        copy_ori_attrs = copy.deepcopy(attrs_dict)
        copy_remove_attrs = copy.deepcopy(remove_attrs_dict)
        # print(copy_ori_attrs)
        # print(copy_remove_attrs)
        # 기존: find_identical_attractors(copy_ori_attrs, remove_attrs_dict, i)
        try: # find_identical_attractors 내부에서 발생하는 예외를 더 상세히 잡기 위함
            find_identical_attractors(copy_ori_attrs, remove_attrs_dict, i)
        except Exception as e:
            print(f"DEBUG: find_identical_attractors 호출 중 예외 발생: {type(e).__name__}: {e}")
            raise # 이 예외를 다시 상위 try-except 블록으로 전달하여, 사용자 코드의 catch 문으로 가게 합니다.
        
        
    except:
        print("--현재 로직은 Phenotype Node 또는 가장 상위 Node 입니다.--")


------remove logic of Node1------
./network/random_network/random_network_1.bnet
ERROR failed to run bnet2primes: cmd=/data/student/anaconda3/envs/python_3_9/lib/python3.9/site-packages/pyboolnet/binaries/BNetToPrime/BNetToPrime_linux64, return_code=1, out=b'{'
{}
--현재 로직은 Phenotype Node 또는 가장 상위 Node 입니다.--
------remove logic of Node10------
./network/random_network/random_network_10.bnet
ERROR failed to run bnet2primes: cmd=/data/student/anaconda3/envs/python_3_9/lib/python3.9/site-packages/pyboolnet/binaries/BNetToPrime/BNetToPrime_linux64, return_code=1, out=b'{'
{}
--현재 로직은 Phenotype Node 또는 가장 상위 Node 입니다.--
------remove logic of Node11------
./network/random_network/random_network_11.bnet
{}
WARNING The state transition graph will consist of up to 2**30=1073741824 states, depending on the initial states.

--- [Simulation Function] Detected Attractors for Current Perturbation ---
  > Attractor ID 0 (Steady State, Basin 21.14%)
    State Str:  100111100110000100000000010010
  > At